In [1]:
import pandas as pd

In [2]:
df_train = pd.read_csv('./dataset/train_ts_feature_added.csv')
df_train['datetime_iso'] = pd.to_datetime(df_train['datetime_iso'])

df_test = pd.read_csv('./dataset/test_ts_feature_added.csv')
df_test['datetime_iso'] = pd.to_datetime(df_test['datetime_iso'])
df_test_date = df_test['datetime_iso']

In [3]:
df_train.head()

,datetime_iso,temp,d_point,feels,min_temp,max_temp,prssr,hum,wind_spd,wind_deg,rain_1h,clouds,day_of_the_week,month,season,yearly_mean,monthly_mean,weekly_mean
0,1979-01-01 00:00:00+00:00,24.75,23.89,25.76,24.28,25.22,1012.0,95.0,0.82,320.0,0.00,100.0,0,1,1,0.492286,0.442561,0.485107
1,1979-01-01 01:00:00+00:00,24.58,23.73,25.57,23.99,25.26,1012.0,95.0,0.96,338.0,0.00,100.0,0,1,1,0.492286,0.442561,0.485107
2,1979-01-01 02:00:00+00:00,26.60,24.06,26.60,26.10,27.39,1012.0,86.0,1.22,339.0,0.00,99.0,0,1,1,0.492286,0.442561,0.485107
3,1979-01-01 03:00:00+00:00,27.31,24.37,30.90,26.59,28.36,1012.0,84.0,1.08,342.0,0.13,94.0,0,1,1,0.492286,0.442561,0.485107
4,1979-01-01 04:00:00+00:00,27.41,25.05,31.54,26.58,28.31,1011.0,87.0,0.86,336.0,0.34,100.0,0,1,1,0.492286,0.442561,0.485107


## Modelling using Pycaret:Catboost

In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341880 entries, 0 to 341879
Data columns (total 18 columns):
 #   Column           Non-Null Count   Dtype              
---  ------           --------------   -----              
 0   datetime_iso     341880 non-null  datetime64[ns, UTC]
 1   temp             341880 non-null  float64            
 2   d_point          341880 non-null  float64            
 3   feels            341880 non-null  float64            
 4   min_temp         341880 non-null  float64            
 5   max_temp         341880 non-null  float64            
 6   prssr            341880 non-null  float64            
 7   hum              341880 non-null  float64            
 8   wind_spd         341880 non-null  float64            
 9   wind_deg         341880 non-null  float64            
 10  rain_1h          341880 non-null  float64            
 11  clouds           341880 non-null  float64            
 12  day_of_the_week  341880 non-null  int64              
 13 

In [5]:
df_train = df_train.drop(['datetime_iso'],axis = 1)
df_test = df_test.drop(['datetime_iso','rain_1h'],axis = 1)

In [6]:
from pycaret.regression import *
s = setup(df_train, target = 'rain_1h', session_id = 43)

,Description,Value
0,Session id,43
1,Target,rain_1h
2,Target type,Regression
3,Original data shape,"(341880, 17)"
4,Transformed data shape,"(341880, 17)"
5,Transformed train set shape,"(239315, 17)"
6,Transformed test set shape,"(102565, 17)"
7,Numeric features,16
8,Preprocess,True
9,Imputation type,simple


In [7]:
best = compare_models()

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,22:22:30
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Random Forest Regressor


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lr,Linear Regression,0.4903,0.8313,0.9116,0.0619,0.3908,1.1178,0.3810
ridge,Ridge Regression,0.4903,0.8313,0.9116,0.0619,0.3908,1.1178,0.0570
br,Bayesian Ridge,0.4902,0.8313,0.9116,0.0619,0.3908,1.1175,0.0880
en,Elastic Net,0.5090,0.8706,0.9329,0.0176,0.3987,0.9197,0.0560
omp,Orthogonal Matching Pursuit,0.5134,0.8854,0.9408,0.0008,0.4043,0.8723,0.0540
lasso,Lasso Regression,0.5133,0.8855,0.9409,0.0007,0.4042,0.8684,0.1500
llar,Lasso Least Angle Regression,0.5133,0.8855,0.9409,0.0007,0.4042,0.8684,0.0520
par,Passive Aggressive Regressor,0.4328,0.8942,0.9454,-0.0091,0.3874,1.0051,0.1050
knn,K Neighbors Regressor,0.4806,0.9164,0.9571,-0.0343,0.4206,1.3603,3.5980
huber,Huber Regressor,0.3834,0.9348,0.9667,-0.0550,0.3942,0.7031,1.4600


Processing:   0%|          | 0/85 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [8]:
catboost = create_model('catboost')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.4460,0.7220,0.8497,0.1401,0.3671,1.1837
1,0.4430,0.7407,0.8607,0.1555,0.3636,1.1959
2,0.4571,0.7713,0.8783,0.1529,0.3706,1.1905
3,0.4553,0.7573,0.8702,0.1515,0.3703,1.2153
4,0.4460,0.7632,0.8736,0.1499,0.3657,1.1953
5,0.4460,0.7219,0.8496,0.1413,0.3667,1.1881
6,0.4563,0.7838,0.8853,0.1490,0.3706,1.2176
7,0.4581,0.8195,0.9053,0.1382,0.3722,1.2147
8,0.4463,0.7265,0.8523,0.1496,0.3661,1.1734


In [9]:
predictions = predict_model(catboost, data=df_test)
predictions.head()

,temp,d_point,feels,min_temp,max_temp,prssr,hum,wind_spd,wind_deg,clouds,day_of_the_week,month,season,yearly_mean,monthly_mean,weekly_mean,prediction_label
0,26.59,23.660000,26.590000,26.020000,27.160000,1009.0,84.0,1.45,355.0,97.0,0,1,1,0.492286,0.442561,0.485107,0.347591
1,26.51,24.920000,26.510000,26.059999,28.040001,1009.0,91.0,1.67,351.0,95.0,0,1,1,0.492286,0.442561,0.485107,0.251431
2,28.68,25.709999,34.680000,28.030001,29.299999,1009.0,84.0,1.72,345.0,90.0,0,1,1,0.492286,0.442561,0.485107,1.428462
3,28.84,25.250000,34.509998,28.520000,29.080000,1008.0,81.0,1.49,339.0,91.0,0,1,1,0.492286,0.442561,0.485107,1.311439
4,29.75,24.620001,35.380001,29.309999,30.570000,1007.0,74.0,1.39,339.0,96.0,0,1,1,0.492286,0.442561,0.485107,0.702441


In [10]:
submission = pd.DataFrame()
submission['datetime_iso'] = df_test_date
submission['rain_1h'] = predictions['prediction_label']
submission.head()

,datetime_iso,rain_1h
0,2018-01-01 00:00:00+00:00,0.347591
1,2018-01-01 01:00:00+00:00,0.251431
2,2018-01-01 02:00:00+00:00,1.428462
3,2018-01-01 03:00:00+00:00,1.311439
4,2018-01-01 04:00:00+00:00,0.702441


In [11]:
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49368 entries, 0 to 49367
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   datetime_iso  49368 non-null  datetime64[ns, UTC]
 1   rain_1h       49368 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(1)
memory usage: 771.5 KB


In [12]:
submission.to_csv('./submission/submission_5.csv', index=False)